Image compression using singular value decomposition
=============================


## Low-rank approximation of images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

Let $A\in\mathbb{R}^{n\times m}$ be an arbitrary matrix. Then there exist matrices $U\in\mathbb{R}^{n\times n}$, $D\in\mathbb{R}^{n\times m}$ and $V\in\mathbb{R}^{m\times m}$, such that 

$$ 
A = UDV^*, 
$$ 

where $U$ and $V$ are unitary matrices, that is $U^*U = UU^* = I_n$ and $V^*V=VV^*=I_m$ and $D$ is a diagonal matrix, that is $d_{ij}=0$ if $i\ne j$. The star operation means conjugate transpose, that is $(V^*)_{ij} = \overline V_{ji}$, but since we are dealing with real matrices now, this is the same as the transpose of the matrix. The diagonal elements in $D$ are nonnegative numbers, in decreasing order: $d_{ii} = \sigma_i$, $\sigma_1\geq\sigma_2\geq\ldots\geq\sigma_r > \sigma_{r+1} = \ldots = \sigma_{\min(n,m)} = 0$, where $r$ is the rank of the matrix $A$. These $\sigma$ values in the diagonal of $D$ are called the singular values of $A$.

#### Low-rank approximations

Let $k\in\mathbb{N}$ a given natural number, where $k\leq\text{rank}(A)\leq\min\{n, m\}$. What we look for is a matrix $A_k$ having $\text{rank}(A_k) = k$ which is the best approximation of $A$ among the matrices that have rank equals to $k$. To formulate the low-rank approximation problem, we would like to solve the following minimalization problem: 

$$ 
\left|\left| A - B \right|\right|_F \to \min !\qquad \mbox{ subject to }\quad B\in\mathbb{R}^{n\times m}, \ \text{rank}(B) = k. 
$$ 

Here $\left|\left| X \right|\right|_F$ denotes the Frobenius norm of a matrix $X$ which is the squareroot of the sum of squares of the elements of $X$.

The solution of this problem can be obtained from the SVD-decomposition of $A$. If $A = UDV^*$, then we keep the first $k$ values in $D$ as is and set the subsequent singular values to zero. Let us denote the resulting diagonal matrix by $D_k$. It is easy to see that we only have to keep the first $k$ columns of $U$ and the first $k$ rows of $V$, since their other columns would be multiplied by zeros anyway. 

To sum up, the matrix $A_k := U_kD_kV_k^*$ is the closest matrix to $A$ (in Frobenius norm) having rank $k$, where $U_k$ and $V_k$ consist of the first $k$ columns and rows of $U$ and $V$, respectively.

In [ ]:
# A has rank 2 since its columns are linearly independent, but if A[2,1] would be 4 instead of 3, 
# the rank would be 1.
A = np.array([[1, 2, 0, 0, 2, 3, -1, -2]]).reshape((4, 2))

print(A)

In [ ]:
n, m = A.shape
rank_A = np.linalg.matrix_rank(A)

print(f"number of rows: {n}, number of columns: {m}")
print(f"the rank of A is {rank_A}")

In [ ]:
U, d, V_T = np.linalg.svd(A, full_matrices=True)
print(f"number of singular values: {len(d)}")

In [ ]:
D = np.concatenate((np.diag(d), np.zeros((n - len(d), m))), axis=0)
A_restored = np.matmul(U, np.matmul(D, V_T))

# A_restored = np.dot(U, np.dot(D, V_T))
# A_restored = (U @ D) @ V_T

print(A_restored)

We mentioned that $A$ is "almost" a one-rank matrix, slightly changing one of its element would reduce its rank by $1$. So define a rank one matrix $B$ that has the same elements as $A$ with one exception.

In [ ]:
B = np.array([[1, 2, 0, 0, 2, 4, -1, -2]]).reshape((4, 2))
print(B)

n, m = B.shape
rank_B = np.linalg.matrix_rank(B)

print(f"number of rows: {n}, number of columns: {m}")
print(f"the rank of B is {rank_B}")

distance = np.linalg.norm(A - B, "fro")
print(f'Distance between A and B is: {distance}')

In [ ]:
U_1 = U[:, :rank_B]
D_1 = D[:rank_B, :rank_B]
V_T_1 = V_T[:rank_B, :]

A_1 = np.matmul(U_1, np.matmul(D_1, V_T_1))
print(f"the best (that is the closest) rank 1 approximation of A is \n {A_1}")

dist_from_A_1 = np.linalg.norm(A - A_1, "fro")
print("The Frobenius-norm of the difference matrix A-A_1 is {}.".format(dist_from_A_1))

Images are represented in a rectangular array where each element corresponds to the grayscale value for that pixel. For colored images we have a $3$-dimensional array of size $n\times m\times 3$, where $n$ and $m$ represents the number of pixels vertically and horizontally, respectively, and for each pixel we store the intensity for colors red, green and blue.

What we are going to do is to repeat the low-rank approximation procedure above on a larger matrix, that is, we create the low-rank approximation of a matrix that represents an image for each color separately. The resulting $3$-dimensional array will be a good approximation of the original image, as we will see soon.

In [ ]:
image = np.array(Image.open("images/Castle_hill.jpg"))

In [ ]:
image = image / 255
row, col, _ = image.shape
print("pixels in one channel: {} * {}".format(row, col))

In [ ]:
fig = plt.figure(figsize=(15, 10))
imgplot = plt.imshow(image)
plt.title("Castle Hill, Budapest")
plt.show()

In [ ]:
image_red = image[:, :, 0]
image_green = image[:, :, 1]
image_blue = image[:, :, 2]

In [ ]:
U_r, d_r, V_r = np.linalg.svd(image_red, full_matrices=True)
U_g, d_g, V_g = np.linalg.svd(image_green, full_matrices=True)
U_b, d_b, V_b = np.linalg.svd(image_blue, full_matrices=True)

In [ ]:
k = 50

U_r_k = U_r[:, :k]
V_r_k = V_r[:k, :]
U_g_k = U_g[:, :k]
V_g_k = V_g[:k, :]
U_b_k = U_b[:, :k]
V_b_k = V_b[:k, :]

d_r_k = d_r[:k]
d_g_k = d_g[:k]
d_b_k = d_b[:k]

In [ ]:
image_red_approx = np.matmul(U_r_k, np.matmul(np.diag(d_r_k), V_r_k))
image_green_approx = np.matmul(U_g_k, np.matmul(np.diag(d_g_k), V_g_k))
image_blue_approx = np.matmul(U_b_k, np.matmul(np.diag(d_b_k), V_b_k))

In [ ]:
image_reconstructed = np.zeros((row, col, 3))

image_reconstructed[:, :, 0] = image_red_approx
image_reconstructed[:, :, 1] = image_green_approx
image_reconstructed[:, :, 2] = image_blue_approx

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.imshow(image_reconstructed)
plt.title('Castle hill, compressed image using the best rank-{} approximation'.format(k))
plt.show()

In [ ]:
print(np.max(image_reconstructed))
print(np.min(image_reconstructed))

In [ ]:
image_reconstructed[image_reconstructed < 0] = 0
image_reconstructed[image_reconstructed > 1] = 1

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.imshow(image_reconstructed)
plt.title('Castle hill, compressed image using the best rank-{} approximation'.format(k))
plt.show()

In [ ]:
def compress_an_image(filename, k=50):
    image = np.array(Image.open(filename))
    image = image / 255
    row, col, _ = image.shape
    
    U_r, d_r, V_r = np.linalg.svd(image[:, :, 0], full_matrices=True)
    U_g, d_g, V_g = np.linalg.svd(image[:, :, 1], full_matrices=True)
    U_b, d_b, V_b = np.linalg.svd(image[:, :, 2], full_matrices=True)
    
    U_r_k = U_r[:, :k]
    V_r_k = V_r[:k, :]
    U_g_k = U_g[:, :k]
    V_g_k = V_g[:k, :]
    U_b_k = U_b[:, :k]
    V_b_k = V_b[:k, :]

    d_r_k = d_r[:k]
    d_g_k = d_g[:k]
    d_b_k = d_b[:k]

    image_red_approx = np.matmul(U_r_k, np.matmul(np.diag(d_r_k), V_r_k))
    image_green_approx = np.matmul(U_g_k, np.matmul(np.diag(d_g_k), V_g_k))
    image_blue_approx = np.matmul(U_b_k, np.matmul(np.diag(d_b_k), V_b_k))

    image_reconstructed = np.zeros((row, col, 3))
    image_reconstructed[:, :, 0] = image_red_approx
    image_reconstructed[:, :, 1] = image_green_approx
    image_reconstructed[:, :, 2] = image_blue_approx
    image_reconstructed[image_reconstructed < 0] = 0
    image_reconstructed[image_reconstructed > 1] = 1
    return image, image_reconstructed

In [ ]:
image, image_compressed = compress_an_image(filename="images/Castle_hill.jpg", k=100)

In [ ]:
fig = plt.figure(figsize=(16, 16))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.subplot(1, 2, 2)
plt.imshow(image_compressed)
plt.show()